## Import modules and data

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
filename = '../data/original/all_hourly_data.h5'
df_pats = pd.read_hdf(filename, 'patients')

In [3]:
multi_to_patid = pd.read_csv('../data/processed/multi_to_patid.csv')
multi_to_patid.set_index(keys=['subject_id','hadm_id','icustay_id'], inplace=True)

## Basic Exploration

In [4]:
df_pats.head()

,,,gender,ethnicity,age,insurance,admittime,diagnosis_at_admission,dischtime,discharge_location,fullcode_first,dnr_first,...,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,readmission_30,max_hours
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,,,,
3,145834,211552,M,WHITE,76.526792,Medicare,2101-10-20 19:08:00,HYPOTENSION,2101-10-31 13:58:00,SNF,1.0,0.0,...,2101-10-26 20:43:09,6.064560,EMERGENCY,MICU,0,0,0,1,0,145
4,185777,294638,F,WHITE,47.845047,Private,2191-03-16 00:28:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",2191-03-23 18:41:00,HOME WITH HOME IV PROVIDR,1.0,0.0,...,2191-03-17 16:46:31,1.678472,EMERGENCY,MICU,0,0,0,1,0,40
6,107064,228232,F,WHITE,65.942297,Medicare,2175-05-30 07:15:00,CHRONIC RENAL FAILURE/SDA,2175-06-15 16:00:00,HOME HEALTH CARE,1.0,0.0,...,2175-06-03 13:39:54,3.672917,ELECTIVE,SICU,0,0,0,1,0,88
9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.790228,Medicaid,2149-11-09 13:06:00,HEMORRHAGIC CVA,2149-11-14 10:15:00,DEAD/EXPIRED,1.0,0.0,...,2149-11-14 20:52:14,5.323056,EMERGENCY,MICU,1,1,1,1,0,127
11,194540,229441,F,WHITE,50.148295,Private,2178-04-16 06:18:00,BRAIN MASS,2178-05-11 19:00:00,HOME HEALTH CARE,1.0,0.0,...,2178-04-17 20:21:05,1.584410,EMERGENCY,SICU,0,0,0,1,0,38


## Isolate Labels

In [5]:
df_labels = df_pats[['mort_hosp']]

## Transfer to singular index

In [6]:
df_labels = df_labels.join(multi_to_patid)
df_labels.set_index('pat_id', inplace=True)

In [12]:
df_labels.mort_hosp = df_labels.mort_hosp.astype('category')

In [13]:
df_labels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34472 entries, 0 to 34471
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   mort_hosp  34472 non-null  category
dtypes: category(1)
memory usage: 303.1 KB


In [ ]:
pd.get_du

## Split & Save Results

In [15]:
processed_dir = '../data/processed/'
csv_filename = 'labels.csv'

In [16]:
train_idxs = np.load(os.path.join(processed_dir, 'train_idxs.npy'))
test_idxs = np.load(os.path.join(processed_dir, 'test_idxs.npy'))

In [17]:
df_train = df_labels.loc[train_idxs].copy()
df_test = df_labels.loc[test_idxs].copy()

In [18]:
df_train.to_csv(os.path.join(processed_dir, 'train/', csv_filename))
df_test.to_csv(os.path.join(processed_dir, 'test/', csv_filename))